In [65]:
import pandas as pd

submission = pd.read_csv("./interim_data/w_submission.csv")

In [66]:
t_detailed = pd.read_csv("./data/WNCAATourneyDetailedResults.csv")
t_detailed.head()
t_seeds = pd.read_csv("./data/WNCAATourneySeeds.csv")
t_seeds.head()
t_slots = pd.read_csv("./data/WNCAATourneySlots.csv")
t_slots.head()
s_detailed = pd.read_csv("./data/WRegularSeasonDetailedResults.csv")
s_detailed.head()
w_teams = pd.read_csv("./data/WTeams.csv")
w_teams.head()
save_t_detailed = t_detailed.copy()
w_conferences = pd.read_csv("./data/WTeamConferences.csv")
w_conferences.head()
w_seasons = pd.read_csv("./data/WSeasons.csv")
w_seasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,10/27/1997,East,Midwest,Mideast,West
1,1999,10/26/1998,East,Mideast,Midwest,West
2,2000,11/01/1999,East,Midwest,Mideast,West
3,2001,10/30/2000,East,Midwest,Mideast,West
4,2002,10/29/2001,East,West,Mideast,Midwest


## Adding Team Names to Tournament Results

In [67]:
t_detailed = save_t_detailed.copy()
t_detailed.head()
t_detailed = t_detailed.merge(w_teams, left_on="WTeamID", right_on="TeamID", how="left")
t_detailed["WTeamName"] = t_detailed["TeamName"]
t_detailed.drop(columns=["TeamID", "TeamName"], inplace=True)
t_detailed = t_detailed.merge(w_teams, left_on="LTeamID", right_on="TeamID", how="left")
t_detailed["LTeamName"] = t_detailed["TeamName"]
t_detailed.drop(columns=["TeamID", "TeamName"], inplace=True)
t_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WTeamName,LTeamName
0,2010,138,3124,69,3201,55,N,0,28,57,...,5,17,19,12,18,4,1,18,Baylor,Fresno St
1,2010,138,3173,67,3395,66,N,0,23,59,...,15,18,26,8,8,8,6,22,Dayton,TCU
2,2010,138,3181,72,3214,37,H,0,26,57,...,8,10,21,4,16,6,4,20,Duke,Hampton
3,2010,138,3199,75,3256,61,H,0,25,63,...,22,16,21,13,16,5,4,24,Florida St,Louisiana Tech
4,2010,138,3207,62,3265,42,N,0,24,68,...,17,16,22,9,10,3,4,12,Georgetown,Marist


## Formatting to similar submission format

In [68]:
cols = ["Season", "first_id", "second_id"]
t_detailed["first_id"] = t_detailed[['WTeamID','LTeamID']].min(axis=1)
t_detailed["second_id"] = t_detailed[['WTeamID','LTeamID']].max(axis=1)
t_detailed["prob"] = 0
t_detailed.loc[t_detailed.first_id == t_detailed.WTeamID, "prob"] = 1
t_detailed["t_id"] = t_detailed[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
t_detailed.drop(columns=["first_id", "second_id"], inplace=True)

## Adding Seeds to the data

In [69]:
seeded_t_detailed = t_detailed.copy()
seeded_t_detailed = pd.merge(t_detailed, t_seeds, how="left", left_on=["WTeamID", "Season"], right_on=["TeamID", "Season"])
seeded_t_detailed["winning_seed"] = seeded_t_detailed["Seed"].str.extract('(\d+)').astype("int32")
seeded_t_detailed["winning_region_id"] = seeded_t_detailed["Seed"].str.slice(0,1)
seeded_t_detailed.drop(columns=["TeamID", "Seed"], inplace=True)
seeded_t_detailed = pd.merge(seeded_t_detailed, t_seeds, how="left", left_on=["LTeamID", "Season"], right_on=["TeamID", "Season"])
seeded_t_detailed["losing_seed"] = seeded_t_detailed["Seed"].str.extract('(\d+)').astype("int32")
seeded_t_detailed["losing_region_id"] = seeded_t_detailed["Seed"].str.slice(0,1)
seeded_t_detailed.drop(columns=["TeamID", "Seed"], inplace=True)
seeded_t_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LBlk,LPF,WTeamName,LTeamName,prob,t_id,winning_seed,winning_region_id,losing_seed,losing_region_id
0,2010,138,3124,69,3201,55,N,0,28,57,...,1,18,Baylor,Fresno St,1,2010_3124_3201,4,X,13,X
1,2010,138,3173,67,3395,66,N,0,23,59,...,6,22,Dayton,TCU,1,2010_3173_3395,8,X,9,X
2,2010,138,3181,72,3214,37,H,0,26,57,...,4,20,Duke,Hampton,1,2010_3181_3214,2,X,15,X
3,2010,138,3199,75,3256,61,H,0,25,63,...,4,24,Florida St,Louisiana Tech,1,2010_3199_3256,3,W,14,W
4,2010,138,3207,62,3265,42,N,0,24,68,...,4,12,Georgetown,Marist,1,2010_3207_3265,5,X,12,X


## Adding Conferences to the data

In [70]:
sc_t_detailed = pd.merge(seeded_t_detailed, w_conferences, left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"], how="left")
sc_t_detailed["win_conf"] = sc_t_detailed["ConfAbbrev"]
sc_t_detailed.drop(columns=["TeamID", "ConfAbbrev"], inplace=True)
sc_t_detailed = pd.merge(sc_t_detailed, w_conferences, left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"], how="left")
sc_t_detailed["loss_conf"] = sc_t_detailed["ConfAbbrev"]
sc_t_detailed.drop(columns=["TeamID", "ConfAbbrev"], inplace=True)
sc_t_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WTeamName,LTeamName,prob,t_id,winning_seed,winning_region_id,losing_seed,losing_region_id,win_conf,loss_conf
0,2010,138,3124,69,3201,55,N,0,28,57,...,Baylor,Fresno St,1,2010_3124_3201,4,X,13,X,big_twelve,wac
1,2010,138,3173,67,3395,66,N,0,23,59,...,Dayton,TCU,1,2010_3173_3395,8,X,9,X,a_ten,mwc
2,2010,138,3181,72,3214,37,H,0,26,57,...,Duke,Hampton,1,2010_3181_3214,2,X,15,X,acc,meac
3,2010,138,3199,75,3256,61,H,0,25,63,...,Florida St,Louisiana Tech,1,2010_3199_3256,3,W,14,W,acc,wac
4,2010,138,3207,62,3265,42,N,0,24,68,...,Georgetown,Marist,1,2010_3207_3265,5,X,12,X,big_east,maac


## Adding in the tournament Regions

In [71]:
regions = w_seasons.copy()
regions.drop(columns=["DayZero"], inplace=True)
vert_regions = regions.melt(id_vars=["Season"])
vert_regions.variable = vert_regions.variable.str.strip("Region")
vert_regions.head()
vert_regions.columns = ["Season", "ID", "Region"]

In [74]:
scr_t_detailed = sc_t_detailed.copy()
scr_t_detailed = pd.merge(scr_t_detailed, vert_regions, left_on=["Season", "winning_region_id"], right_on=["Season", "ID"])
scr_t_detailed["winning_region"] = scr_t_detailed["Region"]
scr_t_detailed.drop(columns=["ID", "Region"], inplace=True)
scr_t_detailed = pd.merge(scr_t_detailed, vert_regions, left_on=["Season", "losing_region_id"], right_on=["Season", "ID"])
scr_t_detailed["losing_region"] = scr_t_detailed["Region"]
scr_t_detailed.drop(columns=["ID", "Region"], inplace=True)

scr_t_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,prob,t_id,winning_seed,winning_region_id,losing_seed,losing_region_id,win_conf,loss_conf,winning_region,losing_region
0,2010,138,3124,69,3201,55,N,0,28,57,...,1,2010_3124_3201,4,X,13,X,big_twelve,wac,Memphis,Memphis
1,2010,138,3173,67,3395,66,N,0,23,59,...,1,2010_3173_3395,8,X,9,X,a_ten,mwc,Memphis,Memphis
2,2010,138,3181,72,3214,37,H,0,26,57,...,1,2010_3181_3214,2,X,15,X,acc,meac,Memphis,Memphis
3,2010,138,3199,75,3256,61,H,0,25,63,...,1,2010_3199_3256,3,W,14,W,acc,wac,Dayton,Dayton
4,2010,138,3207,62,3265,42,N,0,24,68,...,1,2010_3207_3265,5,X,12,X,big_east,maac,Memphis,Memphis


In [77]:
scr_t_detailed.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'WTeamName', 'LTeamName', 'prob', 't_id', 'winning_seed',
       'winning_region_id', 'losing_seed', 'losing_region_id', 'win_conf',
       'loss_conf', 'winning_region', 'losing_region'],
      dtype='object')

## We did all of that cleaning, so let's now get rid of it :3

In [81]:
s_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14


In [82]:
cols = ["Season", "first_id", "second_id"]
s_detailed["first_id"] = s_detailed[['WTeamID','LTeamID']].min(axis=1)
s_detailed["second_id"] = s_detailed[['WTeamID','LTeamID']].max(axis=1)
s_detailed["prob"] = 0
s_detailed.loc[s_detailed.first_id == s_detailed.WTeamID, "prob"] = 1
s_detailed["t_id"] = s_detailed[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
s_detailed.drop(columns=["first_id", "second_id"], inplace=True)

In [83]:
c_s_detailed = pd.merge(s_detailed, w_conferences, left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"], how="left")
c_s_detailed["win_conf"] = c_s_detailed["ConfAbbrev"]
c_s_detailed.drop(columns=["TeamID", "ConfAbbrev"], inplace=True)
c_s_detailed = pd.merge(c_s_detailed, w_conferences, left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"], how="left")
c_s_detailed["loss_conf"] = c_s_detailed["ConfAbbrev"]
c_s_detailed.drop(columns=["TeamID", "ConfAbbrev"], inplace=True)
c_s_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LDR,LAst,LTO,LStl,LBlk,LPF,prob,t_id,win_conf,loss_conf
0,2010,11,3103,63,3237,49,H,0,23,54,...,27,11,23,7,6,19,1,2010_3103_3237,mac,summit
1,2010,11,3104,73,3399,68,N,0,26,62,...,26,7,20,4,2,27,1,2010_3104_3399,sec,ovc
2,2010,11,3110,71,3224,59,A,0,29,62,...,23,8,15,6,0,15,1,2010_3110_3224,patriot,meac
3,2010,11,3111,63,3267,58,A,0,27,52,...,22,15,11,14,5,14,1,2010_3111_3267,southern,cusa
4,2010,11,3119,74,3447,70,H,1,30,74,...,32,12,14,4,2,14,1,2010_3119_3447,patriot,nec


In [87]:
sd = c_s_detailed.copy()
sd["first_conf"] = sd.win_conf
sd.loc[sd.prob == 0, "first_conf"] = sd.loss_conf
sd["second_conf"] = sd.loss_conf
sd.loc[sd.prob == 0, "first_conf"] = sd.win_conf
sd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LTO,LStl,LBlk,LPF,prob,t_id,win_conf,loss_conf,first_conf,second_conf
0,2010,11,3103,63,3237,49,H,0,23,54,...,23,7,6,19,1,2010_3103_3237,mac,summit,mac,summit
1,2010,11,3104,73,3399,68,N,0,26,62,...,20,4,2,27,1,2010_3104_3399,sec,ovc,sec,ovc
2,2010,11,3110,71,3224,59,A,0,29,62,...,15,6,0,15,1,2010_3110_3224,patriot,meac,patriot,meac
3,2010,11,3111,63,3267,58,A,0,27,52,...,11,14,5,14,1,2010_3111_3267,southern,cusa,southern,cusa
4,2010,11,3119,74,3447,70,H,1,30,74,...,14,4,2,14,1,2010_3119_3447,patriot,nec,patriot,nec


In [93]:
only_conf = sd[["first_conf", "second_conf", "t_id", "prob"]]
only_conf.head()

only_conf_d = pd.get_dummies(only_conf, columns=["first_conf", "second_conf"], dtype="int32")

X = only_conf_d.drop(columns = ["t_id", "prob"])
X.head()
y = only_conf_d["prob"]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

lr = LogisticRegression()
lr.fit(X_train, y_train)

preds = lr.predict_proba(X_test)[:,1]

array([0.52025572, 0.45561104, 0.50489249, ..., 0.56501208, 0.54962679,
       0.58212372], shape=(16262,))

In [ ]:
# Goal: Denormalize data so that it is teamID, season, daynum, win/loss, score, season point diff, etc


## Modeling Ideas:

Main thing to predict is probability of a win


This would be a great logistic regression problem, we can also think about an NN with two output nodes for each teams likelyhood of winning. This would also be a good chance to look into Boosting algs (CatBoost, XGBoost, LightGBM). Something with Autogluon?

